<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Tornato</font></h1>

In below code I will be procesing data.

In [96]:
import requests
import pandas as pd
import numpy as np
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(My_table))[0]
indexNames = df[ df['Borough'] == 'Not assigned' ].index
df.drop(indexNames , inplace=True)
for index, row in df.iterrows():
    if row['Neighbourhood']=='Not assigned':
        df.replace(row['Neighbourhood'], row['Borough'])

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
df.shape

(211, 3)

Load the csv file to get longitude & Latitude of postal code of Tornato

In [40]:
!wget -q -O 'geo_data.csv' http://cocl.us/Geospatial_data

In [97]:
loc_info = pd.read_csv('geo_data.csv')
loc_info.rename(columns={'Postal Code' : 'Postcode'}, inplace=True)
loc_info.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [98]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornato are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornato are 43.653963, -79.387207.


In [99]:
toranto_data=pd.merge(df,loc_info[['Postcode','Latitude','Longitude']],on='Postcode')
toranto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [100]:
# create map of Tornati using latitude and longitude values
import folium

map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toranto_data['Latitude'], toranto_data['Longitude'], toranto_data['Borough'], toranto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [101]:
CLIENT_ID = '0BLT5PUIYRHFVZUYVVR2A5VN3W4MMJG4UHTZQM4G3I15L0QJ' # your Foursquare ID
CLIENT_SECRET = 'PY2ZZPBSJUVQF2JYR0IIAR0QJYVEHKG3TZ3YXQRJ1XQ2HKUT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0BLT5PUIYRHFVZUYVVR2A5VN3W4MMJG4UHTZQM4G3I15L0QJ
CLIENT_SECRET:PY2ZZPBSJUVQF2JYR0IIAR0QJYVEHKG3TZ3YXQRJ1XQ2HKUT


In [102]:
neighborhood_latitude = toranto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toranto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toranto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [105]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0BLT5PUIYRHFVZUYVVR2A5VN3W4MMJG4UHTZQM4G3I15L0QJ&client_secret=PY2ZZPBSJUVQF2JYR0IIAR0QJYVEHKG3TZ3YXQRJ1XQ2HKUT&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [106]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d271949429bfc0025c932be'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [107]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [108]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [110]:
tornato_venues = getNearbyVenues(names=toranto_data['Neighbourhood'],
                                   latitudes=toranto_data['Latitude'],
                                   longitudes=toranto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Not assigned
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

In [111]:
print(tornato_venues.shape)
tornato_venues.head()

(4469, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [112]:
tornato_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Albion Gardens,11,11,11,11,11,11
Alderwood,11,11,11,11,11,11
Bathurst Manor,18,18,18,18,18,18
Bathurst Quay,15,15,15,15,15,15
Bayview Village,4,4,4,4,4,4
Beaumond Heights,11,11,11,11,11,11


In [113]:
# one hot encoding
tornato_onehot = pd.get_dummies(tornato_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tornato_onehot['Neighborhood'] = tornato_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tornato_onehot.columns[-1]] + list(tornato_onehot.columns[:-1])
tornato_onehot = tornato_onehot[fixed_columns]

tornato_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
tornato_grouped = tornato_onehot.groupby('Neighborhood').mean().reset_index()
tornato_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.055556,0.000000,0.0000,0.000000,0.000000,0.000000
6,Bathurst Quay,0.000000,0.0,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000


In [115]:
num_top_venues = 5

for hood in tornato_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tornato_grouped[tornato_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Sandwich Place  0.25
3    Skating Rink  0.25
4     Yoga Studio  0.00


----Agincourt North----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  Miscellaneous Shop   0.0
3              Museum   0.0
4       Movie Theater   0.0


----Albion Gardens----
                 venue  freq
0        Grocery Store  0.18
1          Pizza Place  0.18
2           Beer Store  0.09
3  Fried Chicken Joint  0.09
4         Liquor Store  0.09


----Alderwood----
                venue  freq
0         Pizza Place  0.18
1        Skating Rink  0.09
2  Athletics & Sports  0.09
3         Coffee Shop  0.09
4                 Pub  0.09


----Bathurst Manor----
                       venue  freq
0                Coffee Shop

In [116]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [117]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tornato_grouped['Neighborhood']

for ind in np.arange(tornato_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tornato_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,American Restaurant,Cosmetics Shop,Hotel,Gym,Bakery
1,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Women's Store,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,Agincourt North,Playground,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,Albion Gardens,Pizza Place,Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Sandwich Place,Liquor Store,Beer Store,Pharmacy,Gluten-free Restaurant
4,Alderwood,Pizza Place,Gym,Pharmacy,Dance Studio,Skating Rink,Coffee Shop,Sandwich Place,Pool,Athletics & Sports,Pub


In [118]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

tornato_grouped_clustering = tornato_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tornato_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [119]:
# add clustering labels
import numpy as np
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tornato_merged = toranto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tornato_merged = tornato_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
tornato_merged['Cluster Labels'] = tornato_merged['Cluster Labels'].replace(np.nan, 0)
tornato_merged = tornato_merged.astype({"Cluster Labels": int})
tornato_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Hockey Arena,Grocery Store,Portuguese Restaurant,Coffee Shop,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Mexican Restaurant,Breakfast Spot,Theater,Café,Gym / Fitness Center,Pub,Restaurant
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Mexican Restaurant,Breakfast Spot,Theater,Café,Gym / Fitness Center,Pub,Restaurant
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,1,Clothing Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Cosmetics Shop
5,M6A,North York,Lawrence Manor,43.718518,-79.464763,1,Clothing Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Cosmetics Shop
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494,1,Coffee Shop,Gym,Diner,Park,Italian Restaurant,Portuguese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M1B,Scarborough,Rouge,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store
9,M1B,Scarborough,Malvern,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [120]:
# create map
tornato_merged = tornato_merged.astype({"Cluster Labels": int})
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tornato_merged['Latitude'], tornato_merged['Longitude'], tornato_merged['Neighbourhood'], tornato_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster

Cluster

Cluster

In [123]:
tornato_merged.loc[tornato_merged['Cluster Labels'] == 0, tornato_merged.columns[[1] + list(range(5, tornato_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Diner
7,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,York,0,Field,Hockey Arena,Park,Trail,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
38,York,0,Park,Women's Store,Market,Fast Food Restaurant,Pharmacy,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
60,East York,0,Park,Metro Station,Convenience Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
70,North York,0,Airport,Park,Other Repair Shop,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
71,North York,0,Airport,Park,Other Repair Shop,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
82,North York,0,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
83,North York,0,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
89,North York,0,Construction & Landscaping,Park,Bakery,Basketball Court,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
